# HIVE

## HIVE 系统架构
```
    建立在 Hadoop 上的数据仓库基础架构，ETL 的一种，定义语法 HQL 是一种 SQL 解析引擎，将 SQL 语句转译成 MapReduce Job。
    Hive 表对应 HDFS 上的文件夹，数据对应 HDFS 上的文件。
```
***
- 用户接口： CLI, JDBC/ODBC, WebUI
- 元数据存储： mysql, derby
- 解释器、编译器、优化器、执行器
- Hadoop 的 HDFS 存储， MR 的计算
***

## 配置
``` bash
cd ../conf
mv hive-default.xml.template hive-site.xml
vi hive-site.xml
```

``` xml
<configuration>
    <property>
        <name>javax.jdo.option.ConnectionURL</name>
        <value>jdbc:mysql://ip:3306/hive?createDatabaseIfNotExist=true</value>
    </property>
    <property>
        <name>javax.jdo.option.ConnectionDriverName</name>
        <value>com.mysql.jdbc.Driver</value>
    </property>
        <name>javax.jdo.option.ConnectionUserName</name>
        <value>root</value>
    <property>
        <name>javax.jdo.option.ConnectionPassword</name>
        <value>123</value>
    </property>
</configuration>
```

``` bash
# 进入 hive
bin/hive
```

``` bash
# 无密码进入 beeline
# hadoop 修改配置
vi core-site.xml
```
```
<property>
    <name>hadoop.proxyuser.aurelius.hosts</name>
    <value>*</value>
</property>
<property>
    <name>hadoop.proxyuser.aurelius.groups</name>
    <value>*</value>
</property>
```
``` bash
# 启动元数据服务
bin/hive --service metastore & 
# 启动 hiveserver2
bin/hiveserver2
bin/beeline
beeline> !connect jdbc:hive2://ip:10000
```

## 语法
``` sql
-- 创建表
create table teacher(id bigint, name string) row format delimited fields terminated by '\t';
-- 创建外部表
create external table beauties(id  bigint, name string) row format delimited fields terminated by '\t';
-- 创建分区表
create external table beauties(id bigint, name string) partitioned by (nation string) row format delimited fields terminated by '\t';
-- 查看创建表
show create table teacher;
-- 导入数据
load data local inpath '/root/1.txt' into table teacher;
load data local inpath '/root/a/x' into table beauties partition(nation='China');
-- 修改表分区
-- 在 hdfs手动在表路径建的路径不会自动添加到hive 表分区，须通过alter 语句修改 metastore 中元数据
alter table beauties add partition(nation='Japan' location "/beauty/nation=Japan";
-- 查看数据
select * from teacher;
-- 创建库
create database db1;
```

## sqoop 与 hive
``` bash
./sqoop import --connect jdbc:mysql://ip:3306/db \
--username root --password 123 \
--table beauties --hive-import \
--hive-overwrite --hive-table beauties \
--fields-terminated-by '\t';
```

## hive udf
### udf demo
``` java
package hive.udf;

import org.apache.hadoop.hive.ql.exec.UDF;
import org.apache.hadoop.io.Text;

public class UdfDemo extends UDF {
    public Text evaluate(Text input) {
        String na_e = input.toString();
        String na = naMap.get(na_e);
        if (na == null) {
            na = "test";
        }
        t.set(na);
        return t;
    }

    Text t = new Text();
}
```

### udf 调用过程
``` bash
# 1. 添加 jar 包（在hive 中执行，注意 java 的版本）
hive> add jar /root/UDFDemo.jar;
# 2. 创建临时函数
hive> create temporary function funcname as 'com.edu.hive.udf.UDFDemo';
# 3. 调用
hive> select id, funcname(name) from beauties;
# 4. 将查询结果保存到 HDFS
hive> create table result row format delimited fields terminated by '\t' as \
hive> select id, funcname(name) from beauties;
```

## exception
### ${路径问题}
